In [1]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
dftrain=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') 
dfeval=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
yTrain=dftrain.pop('survived')
yEval=dfeval.pop('survived')

cat=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
num=['age','fare']
fc=[]

for feature_name in cat:
  vocabulary=dftrain[feature_name].unique()
  fc.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in num:
  fc.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(fc)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [17]:
def makeInputFunction(data_df,label_df,num_epochs=32, shuffle=True,batch_size=32):
  def input_function():
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds=ds.shuffle(1000)
    ds=ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn=makeInputFunction(dftrain,yTrain)
eval_input_fn=makeInputFunction(dfeval,yEval,num_epochs=1,shuffle=False)

linear_est=tf.estimator.LinearClassifier(feature_columns=fc)

linear_est.train(train_input_fn) #train
result=linear_est.evaluate(eval_input_fn)

clear_output()
print("Accuracy of the Model : ")
print(result['accuracy'])

Accuracy of the Model : 
0.7689394


In [18]:
result=list(linear_est.predict(eval_input_fn))
print(result[0])

{'logits': array([-2.2555742], dtype=float32), 'logistic': array([0.09486973], dtype=float32), 'probabilities': array([0.9051303 , 0.09486973], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
